<a href="https://colab.research.google.com/github/fxrdhan/Binary-Classification-Exercise/blob/main/Seoul_Bike_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Seoul Bike Sharing Demand Prediction**

**Tugas Besar AI - Linear Regression**

Dataset: [Seoul Bike Sharing Demand](https://archive.ics.uci.edu/dataset/560/seoul+bike+sharing+demand)

Objective: Memprediksi jumlah sepeda yang disewa berdasarkan kondisi cuaca dan waktu

---
## Data Acquisition

In [58]:
!pip install ucimlrepo -q
from ucimlrepo import fetch_ucirepo

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

pd.set_option('display.max_columns', None)

In [59]:
dataset = fetch_ucirepo(id=560)
df = pd.concat([dataset.data.features, dataset.data.targets], axis=1)
df

,Date,Rented Bike Count,Hour,Temperature,Humidity,Wind speed,Visibility,Dew point temperature,Solar Radiation,Rainfall,Snowfall,Seasons,Holiday,Functioning Day
0,1/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,1/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,1/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,1/12/2017,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,1/12/2017,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,30/11/2018,1003,19,4.2,34,2.6,1894,-10.3,0.0,0.0,0.0,Autumn,No Holiday,Yes
8756,30/11/2018,764,20,3.4,37,2.3,2000,-9.9,0.0,0.0,0.0,Autumn,No Holiday,Yes
8757,30/11/2018,694,21,2.6,39,0.3,1968,-9.9,0.0,0.0,0.0,Autumn,No Holiday,Yes
8758,30/11/2018,712,22,2.1,41,1.0,1859,-9.8,0.0,0.0,0.0,Autumn,No Holiday,Yes


---
## Data Understanding

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date                   8760 non-null   object 
 1   Rented Bike Count      8760 non-null   int64  
 2   Hour                   8760 non-null   int64  
 3   Temperature            8760 non-null   float64
 4   Humidity               8760 non-null   int64  
 5   Wind speed             8760 non-null   float64
 6   Visibility             8760 non-null   int64  
 7   Dew point temperature  8760 non-null   float64
 8   Solar Radiation        8760 non-null   float64
 9   Rainfall               8760 non-null   float64
 10  Snowfall               8760 non-null   float64
 11  Seasons                8760 non-null   object 
 12  Holiday                8760 non-null   object 
 13  Functioning Day        8760 non-null   object 
dtypes: float64(6), int64(4), object(4)
memory usage: 958.3+ 

Results:
- 8760 entries, 14 columns
- Tidak ada missing values
- 4 kolom kategorikal (object): Date, Seasons, Holiday, Functioning Day
- 10 kolom numerik: target + 9 features

In [61]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Rented Bike Count,8760.0,704.602055,644.997468,0.0,191.00,504.50,1065.25,3556.00
Hour,8760.0,11.500000,6.922582,0.0,5.75,11.50,17.25,23.00
Temperature,8760.0,12.882922,11.944825,-17.8,3.50,13.70,22.50,39.40
Humidity,8760.0,58.226256,20.362413,0.0,42.00,57.00,74.00,98.00
Wind speed,8760.0,1.724909,1.036300,0.0,0.90,1.50,2.30,7.40
Visibility,8760.0,1436.825799,608.298712,27.0,940.00,1698.00,2000.00,2000.00
Dew point temperature,8760.0,4.073813,13.060369,-30.6,-4.70,5.10,14.80,27.20
Solar Radiation,8760.0,0.569111,0.868746,0.0,0.00,0.01,0.93,3.52
Rainfall,8760.0,0.148687,1.128193,0.0,0.00,0.00,0.00,35.00
Snowfall,8760.0,0.075068,0.436746,0.0,0.00,0.00,0.00,8.80


In [62]:
df.isnull().sum().to_frame('missing_count')

,missing_count
Date,0
Rented Bike Count,0
Hour,0
Temperature,0
Humidity,0
Wind speed,0
Visibility,0
Dew point temperature,0
Solar Radiation,0
Rainfall,0


In [63]:
df[['Seasons', 'Holiday', 'Functioning Day']].apply(lambda x: x.value_counts()).T

,Autumn,Holiday,No,No Holiday,Spring,Summer,Winter,Yes
Seasons,2184.0,NaN,NaN,NaN,2208.0,2208.0,2160.0,NaN
Holiday,NaN,432.0,NaN,8328.0,NaN,NaN,NaN,NaN
Functioning Day,NaN,NaN,295.0,NaN,NaN,NaN,NaN,8465.0


---
## Exploratory Data Analysis (EDA)

In [64]:
# Create subplots
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('Distribution of Rented Bike Count',
                                    'Boxplot of Rented Bike Count'))

# Histogram
fig.add_trace(
    go.Histogram(x=df['Rented Bike Count'], nbinsx=50,
                 marker_color='#0099ff', name='Distribution'),
    row=1, col=1
)

# Boxplot
fig.add_trace(
    go.Box(y=df['Rented Bike Count'], marker_color='#0099ff',
           name='Rented Bike Count'),
    row=1, col=2
)

fig.update_layout(height=400, width=1000, showlegend=False)
fig.update_xaxes(title_text="Rented Bike Count", row=1, col=1)
fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="Rented Bike Count", row=1, col=2)
fig.show()

In [65]:
# Hitung statistik
Q1 = df['Rented Bike Count'].quantile(0.25)
Q3 = df['Rented Bike Count'].quantile(0.75)
IQR = Q3 - Q1
median = df['Rented Bike Count'].median()
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

In [66]:
print(f"Q1 (25%): {Q1}")
print(f"Q3 (75%): {Q3}")
print(f"IQR: {IQR}")
print(f"Batas bawah outlier: {lower_bound}")
print(f"Batas atas outlier: {upper_bound}")

outliers = df[(df['Rented Bike Count'] < lower_bound) | (df['Rented Bike Count'] > upper_bound)]
print(f"\nJumlah outlier: {len(outliers)} ({len(outliers)/len(df)*100:.2f}%)")

Q1 (25%): 191.0
Q3 (75%): 1065.25
IQR: 874.25
Batas bawah outlier: -1120.375
Batas atas outlier: 2376.625

Jumlah outlier: 158 (1.80%)


**Outliers pada Rented Bike Count**

Menggunakan metode IQR, batas atas outlier ditemukan pada nilai ~2300. Data di atas batas ini dapat mengganggu performa model. Sebagai penanganan, nilai `Rented Bike Count` akan dibatasi maksimal sesuai upper bound untuk mengurangi pengaruh outliers.

In [67]:
numeric_cols = df.select_dtypes(include=[np.number]).columns
corr_matrix = df[numeric_cols].corr()

fig = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns,
    y=corr_matrix.index,
    colorscale='RdBu_r',
    zmid=0,
    text=np.round(corr_matrix.values, 2),
    texttemplate='%{text}',
    textfont={"size": 10},
    hoverongaps=False
))

fig.update_layout(
    title='Correlation Matrix',
    width=900,
    height=800,
    xaxis_tickangle=-45
)
fig.show()

**Temperature & Dew Point memiliki korelasi tinggi (0.91)**

Korelasi 0.91 menunjukkan kedua variabel memberikan informasi hampir identik. Menyimpan keduanya akan menyebabkan multikolinearitas yang dapat mengganggu performa model. Cukup pertahankan `Temperature` karena lebih fundamental, sementara `Dew Point` dapat dihapus.

In [68]:
corr_matrix['Rented Bike Count'].sort_values(ascending=False).to_frame('correlation')

,correlation
Rented Bike Count,1.000000
Temperature,0.538558
Hour,0.410257
Dew point temperature,0.379788
Solar Radiation,0.261837
Visibility,0.199280
Wind speed,0.121108
Rainfall,-0.123074
Snowfall,-0.141804
Humidity,-0.199780


In [69]:
top_features = ['Temperature', 'Hour', 'Dew point temperature',
                'Solar Radiation', 'Humidity', 'Rainfall']

fig = make_subplots(rows=2, cols=3,
                    subplot_titles=top_features,
                    vertical_spacing=0.12,
                    horizontal_spacing=0.08)

row_col = [(1,1), (1,2), (1,3), (2,1), (2,2), (2,3)]

for idx, feat in enumerate(top_features):
    r, c = row_col[idx]
    fig.add_trace(
        go.Scatter(
            x=df[feat],
            y=df['Rented Bike Count'],
            mode='markers',
            marker=dict(color='#0099ff', size=5, opacity=0.3),
            name=feat
        ),
        row=r, col=c
    )
    fig.update_xaxes(title_text=feat, row=r, col=c)
    fig.update_yaxes(title_text='Rented Bike Count', row=r, col=c)

fig.update_layout(
    title='Scatter Plots: Features vs Target',
    height=700,
    width=1200,
    showlegend=False
)
fig.show()

In [70]:
fig = make_subplots(rows=1, cols=3,
                    subplot_titles=('Bike Count by Season',
                                    'Bike Count by Holiday',
                                    'Bike Count by Functioning Day'))

# Season boxplot
for season in df['Seasons'].unique():
    fig.add_trace(
        go.Box(y=df[df['Seasons']==season]['Rented Bike Count'],
               name=season, showlegend=False),
        row=1, col=1
    )

# Holiday boxplot
for holiday in df['Holiday'].unique():
    fig.add_trace(
        go.Box(y=df[df['Holiday']==holiday]['Rented Bike Count'],
               name=holiday, showlegend=False),
        row=1, col=2
    )

# Functioning Day boxplot
for func_day in df['Functioning Day'].unique():
    fig.add_trace(
        go.Box(y=df[df['Functioning Day']==func_day]['Rented Bike Count'],
               name=func_day, showlegend=False),
        row=1, col=3
    )

fig.update_layout(height=400, width=1200)
fig.update_yaxes(title_text='Rented Bike Count')
fig.show()

In [84]:
hourly_avg = df.groupby('Hour')['Rented Bike Count'].mean().reset_index()

fig = go.Figure(data=[
    go.Bar(
        x=hourly_avg['Hour'],
        y=hourly_avg['Rented Bike Count'],
        marker_color='#0099ff'
    )
])

fig.update_layout(
    title='Average Bike Rentals by Hour',
    xaxis_title='Hour of Day',
    yaxis_title='Average Rented Bike Count',
    height=400,
    width=900
)
fig.update_xaxes(tickmode='linear', tick0=0, dtick=1)
fig.show()

### Finding & Action

-
- **Outliers ditemukan di atas nilai 2500**: Filter data sehingga Rented Bike Count dibatasi maksimal 2500
- **Rainfall, Snowfall, Wind, dan Solar Radiation memiliki outliers**: Akan diterapkan threshold filtering untuk masing-masing variabel ini
- **Seasons, Holiday, dan Functioning Day merupakan variabel kategorikal**: Semua variabel kategorikal akan di-encode menjadi bentuk numerik
- **Kolom Date tidak relevan untuk model**: Kolom Date akan dihapus dari dataset

---
## Preprocessing

### Encode Kategorikal

In [72]:
season_map = {'Winter': 1, 'Spring': 2, 'Summer': 3, 'Autumn': 4}
holiday_map = {'No Holiday': 0, 'Holiday': 1}
func_map = {'No': 0, 'Yes': 1}

df['Seasons'] = df['Seasons'].map(season_map)
df['Holiday'] = df['Holiday'].map(holiday_map)
df['Functioning Day'] = df['Functioning Day'].map(func_map)

df[['Seasons', 'Holiday', 'Functioning Day']].head(10)

,Seasons,Holiday,Functioning Day
0,1,0,1
1,1,0,1
2,1,0,1
3,1,0,1
4,1,0,1
5,1,0,1
6,1,0,1
7,1,0,1
8,1,0,1
9,1,0,1


### Hapus Outliers

In [73]:
shape_before = df.shape[0]

df = df[
    (df['Rented Bike Count'] <= 2500) &
    (df['Rainfall'] <= 10) &
    (df['Snowfall'] <= 4) &
    (df['Wind speed'] <= 5) &
    (df['Solar Radiation'] <= 3.5)
]

shape_after = df.shape[0]

pd.DataFrame({
    'status': ['before', 'after', 'removed'],
    'rows': [shape_before, shape_after, shape_before - shape_after]
})

,status,rows
0,before,8760
1,after,8587
2,removed,173


### Drop Multicollinearity

In [74]:
df = df.drop(columns=['Dew point temperature', 'Date'])
df.columns.tolist()

['Rented Bike Count',
 'Hour',
 'Temperature',
 'Humidity',
 'Wind speed',
 'Visibility',
 'Solar Radiation',
 'Rainfall',
 'Snowfall',
 'Seasons',
 'Holiday',
 'Functioning Day']

### Prepare X dan y

In [75]:
X = df.drop(columns=['Rented Bike Count'])
y = df['Rented Bike Count']

pd.DataFrame({'set': ['X', 'y'], 'shape': [X.shape, y.shape]})

,set,shape
0,X,"(8587, 11)"
1,y,"(8587,)"


---
## Split Data

### Train Test Split

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pd.DataFrame({
    'set': ['train', 'test'],
    'X_shape': [X_train.shape, X_test.shape],
    'y_shape': [y_train.shape, y_test.shape]
})

,set,X_shape,y_shape
0,train,"(6869, 11)","(6869,)"
1,test,"(1718, 11)","(1718,)"


---
## Linear Regression Model

### Train dan Predict

In [77]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

pd.DataFrame({'actual': y_test[:10].values, 'predicted': y_pred[:10].round(2)})

,actual,predicted
0,859,599.29
1,210,607.07
2,1021,1152.11
3,119,156.12
4,1811,1332.74
5,1956,1419.60
6,333,624.76
7,155,278.26
8,930,894.87
9,83,-223.65


---
## Model Evaluation

###  Metrics

In [78]:
r2 = r2_score(y_test, y_pred)
n, p = X_test.shape
adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

pd.DataFrame({
    'metric': ['R²', 'Adjusted R²', 'MAE', 'MSE', 'RMSE'],
    'value': [round(r2, 4), round(adj_r2, 4), round(mae, 2), round(mse, 2), round(rmse, 2)]
})

,metric,value
0,R²,0.5615
1,Adjusted R²,0.5587
2,MAE,296.2900
3,MSE,150696.1600
4,RMSE,388.2000


### Actual vs Predicted Plot

In [85]:
fig = go.Figure()

# Scatter plot
fig.add_trace(go.Scatter(
    x=y_test,
    y=y_pred,
    mode='markers',
    marker=dict(color='#0099ff', size=8, opacity=0.4),
    name='Predictions'
))

# Perfect prediction line
fig.add_trace(go.Scatter(
    x=[0, 2500],
    y=[0, 2500],
    mode='lines',
    line=dict(color='red', width=2, dash='dash'),
    name='Perfect Prediction'
))

fig.update_layout(
    title=f'Actual vs Predicted (R² = {r2:.4f})',
    xaxis_title='Actual',
    yaxis_title='Predicted',
    height=600,
    width=800,
    showlegend=True
)
fig.show()

### Residual Plot

In [86]:
residuals = y_test - y_pred

fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('Residual Plot', 'Residual Distribution'))

# Residual scatter plot
fig.add_trace(
    go.Scatter(
        x=y_pred,
        y=residuals,
        mode='markers',
        marker=dict(color='#0099ff', size=8, opacity=0.4),
        name='Residuals'
    ),
    row=1, col=1
)

# Zero line
fig.add_hline(y=0, line_dash="dash", line_color="red", row=1, col=1)

# Residual histogram
fig.add_trace(
    go.Histogram(
        x=residuals,
        nbinsx=50,
        marker_color='#0099ff',
        name='Distribution'
    ),
    row=1, col=2
)

fig.update_xaxes(title_text='Predicted', row=1, col=1)
fig.update_yaxes(title_text='Residuals', row=1, col=1)
fig.update_xaxes(title_text='Residuals', row=1, col=2)
fig.update_yaxes(title_text='Count', row=1, col=2)

fig.update_layout(height=400, width=1100, showlegend=False)
fig.show()

---
## Interpretation

### Koefisien

In [81]:
coef_df = pd.DataFrame({
    'feature': X.columns,
    'coefficient': model.coef_
})
coef_df['abs_coef'] = coef_df['coefficient'].abs()
coef_df = coef_df.sort_values('abs_coef', ascending=False).drop(columns='abs_coef')
coef_df

,feature,coefficient
10,Functioning Day,905.485701
6,Rainfall,-116.560630
8,Seasons,113.633232
9,Holiday,-102.327703
5,Solar Radiation,-59.193711
0,Hour,26.062479
1,Temperature,24.342096
3,Wind speed,20.640243
7,Snowfall,12.369881
2,Humidity,-6.975407


### Feature Importance Plot

In [87]:
coef_sorted = coef_df.sort_values('coefficient')

colors = ['#e74c3c' if c < 0 else '#2ecc71' for c in coef_sorted['coefficient']]

fig = go.Figure(data=[
    go.Bar(
        y=coef_sorted['feature'],
        x=coef_sorted['coefficient'],
        orientation='h',
        marker_color=colors
    )
])

fig.add_vline(x=0, line_color='white', line_dash='dash')

fig.update_layout(
    title='Feature Coefficients (Green = Positive, Red = Negative)',
    xaxis_title='Coefficient',
    yaxis_title='',
    height=500,
    width=900
)
fig.show()

### Persamaan Regresi

In [88]:
intercept = model.intercept_
equation = f"Rented Bike Count = {intercept:.2f}"
for feat, coef in zip(X.columns, model.coef_):
    sign = '+' if coef >= 0 else '-'
    equation += f" {sign} {abs(coef):.2f}*{feat}"

print(equation)

Rented Bike Count = -643.10 + 26.06*Hour + 24.34*Temperature - 6.98*Humidity + 20.64*Wind speed - 0.01*Visibility - 59.19*Solar Radiation - 116.56*Rainfall + 12.37*Snowfall + 113.63*Seasons - 102.33*Holiday + 905.49*Functioning Day


Persamaan:

$$\hat{y} = \left\{ (-643.10) + (26.06 \cdot \text{Hour}) + (24.34 \cdot \text{Temperature}) + (-6.98 \cdot \text{Humidity}) + (20.64 \cdot \text{Wind speed}) + (-0.01 \cdot \text{Visibility}) + (-59.19 \cdot \text{Solar Radiation}) + (-116.56 \cdot \text{Rainfall}) + (12.37 \cdot \text{Snowfall}) + (113.63 \cdot \text{Seasons}) + (-102.33 \cdot \text{Holiday}) + (905.49 \cdot \text{Functioning Day}) \right\}$$